In [105]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [5]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [6]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [7]:
import pandas as pd
import numpy as np
import gzip

In [8]:
# load the 2017 data
scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [9]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz',names = col_names ,header = None)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [10]:
chem = pd.read_csv('./dw-data/chem.csv.gz')
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [11]:
scripts_summary = scripts.describe()
scripts_summary = scripts_summary.drop('min')
scripts_summary = scripts_summary.drop('max')
scripts_summary = scripts_summary.drop('count')
scripts_summary

,items,nic,act_cost,quantity
mean,9.133136,73.058915,67.986613,741.329835
std,29.204198,188.070257,174.401703,3665.426958
25%,1.000000,7.800000,7.330000,28.000000
50%,2.000000,22.640000,21.220000,100.000000
75%,6.000000,65.000000,60.670000,350.000000


In [12]:
items = tuple([sum(scripts['items'])] + list(scripts_summary['items']))
quantity = tuple([sum(scripts['quantity'])] + list(scripts_summary['quantity']))
nic = tuple([sum(scripts['nic'])] + list(scripts_summary['nic']))
act_cost = tuple([sum(scripts['act_cost'])] + list(scripts_summary['act_cost']))

In [13]:
summary_stats = [('items', items), ('quantity', quantity), ('nic', nic), ('act_cost', act_cost)]

In [14]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [15]:
items_by_bnfName = scripts.groupby('bnf_name')['items'].sum()
items_by_bnfName.max()

218583

In [16]:
most_common_item = [(items_by_bnfName.idxmax(), items_by_bnfName.max())]

In [17]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [18]:
print(scripts.shape)
print(scripts.nunique())
scripts.tail(10)

(973193, 7)
practice      856
bnf_code     1975
bnf_name    13471
items         743
nic         44989
act_cost    54048
quantity    12361
dtype: int64


,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
973183,H81615,238510085,Coloplast_Brava Mouldable Rings 48mm x 2,1,58.39,54.05,30
973184,H81615,239401097,Hollister_Conform 2 Flat Barrier +Adh 45,1,98.28,90.98,30
973185,H81615,239407096,Dansac_Nova 2 Flng 43mm P/Cut 30mm,1,47.46,43.94,15
973186,H81615,239407098,Dansac_NovaLife 2 Open Pouch Maxi Opqe 4,1,212.04,196.30,120
973187,H81615,239410093,Coloplast_SenSura Flex Drnbl Bag Maxi Op,2,105.02,97.22,60
973188,H81615,239410100,Coloplast_SenSura Mio Flex B/Plt 50mm C/,1,78.92,73.06,20
973189,H81615,239410100,Coloplast_SenSura Mio Flex Clsd Bag Mini,1,106.36,98.46,60
973190,H81615,239410100,Coloplast_SenSura Mio Flex Clsd Bag Midi,1,159.54,147.69,90
973191,H81615,239607096,Dansac_Nova 1 Convex Urost Pouch Clr C/F,2,329.76,305.28,60
973192,H81615,239610096,Coloplast_SenSura Mio Urost Bag Maxi Tra,1,182.48,168.93,30


In [19]:
print(practices.shape)
print(practices.nunique())
practices.head()

(12020, 7)
code         10843
name         10703
addr_1        8898
addr_2        8110
borough       2721
village        458
post_code     8306
dtype: int64


,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [20]:
practices_final = practices.groupby('code')['post_code'].min()
practices_final = pd.DataFrame(practices_final)
practices_final.reset_index(level=0,inplace=True)
print(practices_final.nunique())
practices_final.head()

code         10843
post_code     8137
dtype: int64


,code,post_code
0,A81001,TS18 1HU
1,A81002,TS18 2AW
2,A81003,TS25 1QU
3,A81004,TS1 3BE
4,A81005,TS14 7DJ


In [21]:
scripts_practices = pd.merge(scripts,practices_final, left_on='practice',right_on='code', how='inner')
print(scripts_practices.shape)
print(scripts_practices.nunique())

scripts_practices.head()

(973193, 9)
practice       856
bnf_code      1975
bnf_name     13471
items          743
nic          44989
act_cost     54048
quantity     12361
code           856
post_code      259
dtype: int64


,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,CH44 5UF


In [22]:
# scripts_practices = scripts_practices.drop(['addr_1','addr_2','borough','village'],axis=1)

In [23]:
scripts_practices.head(10)

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,CH44 5UF
5,N85639,0406000T0,Prochlpzine Mal_Tab 5mg,1,0.97,0.91,28,N85639,CH44 5UF
6,N85639,0407010F0,Co-Codamol_Cap 30mg/500mg,1,0.84,0.89,24,N85639,CH44 5UF
7,N85639,0407010F0,Zapain_Tab 30mg/500mg,1,3.03,2.82,100,N85639,CH44 5UF
8,N85639,0407010H0,Paracet_Oral Susp Paed 120mg/5ml,1,0.62,0.69,100,N85639,CH44 5UF
9,N85639,0501011P0,Phenoxymethylpenicillin Pot_Tab 250mg,2,5.94,5.72,160,N85639,CH44 5UF


In [20]:
# #tests
# maxbnf = scripts_practices[scripts_practices['post_code'] =='B11 4BW'].groupby('bnf_name').sum()['items']
# maxone = maxbnf.max()
# postal_total = scripts_practices[scripts_practices['post_code'] =='B11 4BW']['items'].sum()
# maxone/postal_total

In [21]:
postal_totals = scripts_practices.groupby('post_code')['items'].sum()
postal_totals

post_code
B11 4BW     22731
B12 9LP     17073
B18 7AL     20508
B21 9RY     31027
B23 6DJ     28011
            ...  
WS3 3JP     36890
WS9 8AJ     24483
WV13 2DR    36381
YO11 1UB    28507
YO16 4LZ    53847
Name: items, Length: 259, dtype: int64

In [22]:
def findmaxBnf(groupby_object):
    res = groupby_object.groupby('bnf_name')['items'].sum()
    res=res.sort_index()
    return [res.idxmax(),res.max()]
max_bnfs=scripts_practices.groupby('post_code').apply(findmaxBnf)
max_bnfs

post_code
B11 4BW     [Salbutamol_Inha 100mcg (200 D) CFF, 706]
B12 9LP                      [Paracet_Tab 500mg, 425]
B18 7AL     [Salbutamol_Inha 100mcg (200 D) CFF, 556]
B21 9RY               [Metformin HCl_Tab 500mg, 1033]
B23 6DJ       [Lansoprazole_Cap 30mg (E/C Gran), 599]
                              ...                    
WS3 3JP                      [Paracet_Tab 500mg, 789]
WS9 8AJ                     [Amlodipine_Tab 5mg, 396]
WV13 2DR    [Salbutamol_Inha 100mcg (200 D) CFF, 809]
YO11 1UB               [Omeprazole_Cap E/C 20mg, 847]
YO16 4LZ              [Aspirin Disper_Tab 75mg, 1194]
Length: 259, dtype: object

In [23]:
max_bnfs_frame = pd.DataFrame({'postal_totals':postal_totals,'max_bnfs':max_bnfs})
max_bnfs_frame = max_bnfs_frame.sort_index()
max_bnfs_frame.reset_index(level=0,inplace=True)
max_bnfs_frame.head()

,post_code,postal_totals,max_bnfs
0,B11 4BW,22731,"[Salbutamol_Inha 100mcg (200 D) CFF, 706]"
1,B12 9LP,17073,"[Paracet_Tab 500mg, 425]"
2,B18 7AL,20508,"[Salbutamol_Inha 100mcg (200 D) CFF, 556]"
3,B21 9RY,31027,"[Metformin HCl_Tab 500mg, 1033]"
4,B23 6DJ,28011,"[Lansoprazole_Cap 30mg (E/C Gran), 599]"


In [24]:
max_bnfs_frame[['max_bnf','max_item']] =max_bnfs_frame['max_bnfs'].apply(pd.Series)
max_bnfs_frame['proportion'] = max_bnfs_frame['max_item']/max_bnfs_frame['postal_totals']
max_bnfs_frame =max_bnfs_frame.drop(['max_bnfs'],axis=1)


In [25]:
max_bnfs_frame

,post_code,postal_totals,max_bnf,max_item,proportion
0,B11 4BW,22731,Salbutamol_Inha 100mcg (200 D) CFF,706,0.031059
1,B12 9LP,17073,Paracet_Tab 500mg,425,0.024893
2,B18 7AL,20508,Salbutamol_Inha 100mcg (200 D) CFF,556,0.027111
3,B21 9RY,31027,Metformin HCl_Tab 500mg,1033,0.033294
4,B23 6DJ,28011,Lansoprazole_Cap 30mg (E/C Gran),599,0.021384
5,B61 0AZ,33225,Omeprazole_Cap E/C 20mg,954,0.028713
6,B70 7AW,38789,Paracet_Tab 500mg,975,0.025136
7,B72 1RL,28326,Omeprazole_Cap E/C 20mg,573,0.020229
8,B8 1RZ,17941,Metformin HCl_Tab 500mg,383,0.021348
9,B9 5PU,36212,Ventolin_Evohaler 100mcg (200 D),899,0.024826


In [26]:
# max_bnfs_frame = max_bnfs_frame.drop([289])
# #max_bnfs_frame.sort_index()
# max_bnfs_frame

In [26]:
postal_codes = list(max_bnfs_frame['post_code'])
bnf_names = list(max_bnfs_frame['max_bnf'])
proportions = list(max_bnfs_frame['proportion'])
final_result = zip(postal_codes[:100],bnf_names[:100],proportions[:100])
final_list = list(final_result)


In [27]:
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100
items_by_region =final_list


In [28]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [29]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [30]:
scripts.shape

(973193, 7)

In [31]:
scripts_cpy=scripts.copy()
# scripts_cpy.drop_duplicates(inplace=True)
print(scripts_cpy.shape,scripts.shape)

(973193, 7) (973193, 7)


In [32]:

# isOpioids = [False]*3487
# for opioid in opioids:
#     isOpioids=isOpioids | chem['NAME'].str.lower().str.contains(opioid)
# chem['isOpioids'] = isOpioids
# chem[chem['isOpioids']].count()
chem_cpy=chem.copy()
chem_cpy['isOpioids']= chem_cpy['NAME'].str.lower().str.contains('|'.join(opioids), case=False)
chem_cpy.drop_duplicates(subset='CHEM SUB',inplace=True)
print(chem_cpy.shape,chem.shape)

(3481, 3) (3487, 2)


In [33]:
scripts_chems = pd.merge(scripts_cpy,chem_cpy,left_on='bnf_code',right_on='CHEM SUB',how='left')
scripts_chems['isOpioids']=scripts_chems['isOpioids'].fillna(False)
# scripts_chems.drop_duplicates(inplace=True)
print(scripts_chems[scripts_chems['isOpioids']].shape)
print(scripts_chems.shape)

(34843, 10)
(973193, 10)


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [34]:
opioids_per_practice = scripts_chems.groupby('practice')['isOpioids'].mean()
# SD_ofEach=scripts_chems.groupby('practice')['isOpioids'].std()
SD=scripts_chems['isOpioids'].std()
SD

0.18579817605238425

In [35]:
practices_count=scripts_chems.groupby('practice')['isOpioids'].count()

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [36]:
relative_opioids_per_practice = opioids_per_practice - scripts_chems['isOpioids'].mean()

In [102]:
# relative_opioids_per_practice

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [37]:
count_sqrt = practices_count**(1/2)
count_sqrt

practice
A81005    38.820098
A81007    38.131352
A81011    39.597980
A81012    36.496575
A81017    46.368092
            ...    
Y05570     4.690416
Y05583     5.291503
Y05597     4.242641
Y05660    18.627936
Y05670     3.464102
Name: isOpioids, Length: 856, dtype: float64

In [38]:
sem_of_practices = SD/count_sqrt

In [39]:
standard_error_per_practice = sem_of_practices
opioid_scores = relative_opioids_per_practice/standard_error_per_practice

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [40]:
# zscore_df=pd.merge(opioid_scores,practices_count,left_index=True,right_index=True)
practices_names = practices.groupby('code')['name'].min()
zscore_df = pd.DataFrame({'z-score':opioid_scores,'number_of_scripts':practices_count})
zscore_df = pd.merge(zscore_df,practices_names,left_index=True,right_index=True)
zscore_df.sort_values('z-score',axis=0,ascending=False,inplace=True)
zscore_df.head()

,z-score,number_of_scripts,name
Y01852,11.695818,7,NATIONAL ENHANCED SERVICE
Y03006,7.339043,2,OUTREACH SERVICE NH / RH
Y03668,6.150582,60,BRISDOC HEALTHCARE SERVICES OOH
G81703,5.123032,36,H&R P C SPECIAL SCHEME
Y04997,4.958866,321,HMR BARDOC OOH


In [41]:
final_results = list(zip(zscore_df['name'],zscore_df['z-score'],zscore_df['number_of_scripts']))


In [42]:
unique_practices = 0
# anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100
anomalies = final_results[:100]

In [44]:
#results.head()
#anomalies

In [43]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [44]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

In [45]:
print(scripts16.shape)
print(scripts['bnf_name'].nunique())
print(scripts16['bnf_name'].nunique())

(973193, 7)
13471
13540


In [46]:
items_total_2017=scripts.groupby('bnf_name').agg({'bnf_name':'count'})
items_total_2016=scripts16.groupby('bnf_name').agg({'bnf_name':'count'})
comparator_df = pd.merge(items_total_2017,items_total_2016,left_index=True,right_index=True,how='inner')
comparator_df.rename(columns={'bnf_name_x':'2017_total','bnf_name_y':'2016_total'},inplace=True)
comparator_df['script_growth'] = (comparator_df['2017_total']-comparator_df['2016_total'])/comparator_df['2016_total']
comparator_df.sort_values(by=['script_growth'],inplace=True,ascending=False)
comparator_df.reset_index(level=0,inplace=True)

In [47]:
filtered_df = comparator_df[comparator_df['2016_total']>=50]
filtered_df.head()

,bnf_name,2017_total,2016_total,script_growth
142,Butec_Transdermal Patch 5mcg/hr,277,62,3.467742
228,Butec_Transdermal Patch 10mcg/hr,276,69,3.000000
605,Fostair NEXThaler_Inh 200mcg/6mcg (120D),209,86,1.430233
653,Pneumococcal_Vac 0.5ml Vl (23 Valent),438,193,1.269430
654,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,118,52,1.269231


In [48]:
largest_growth = filtered_df.head(50)

In [49]:
largest_shrinkage = filtered_df.tail(50)

In [50]:
first_50_result=list(zip(largest_growth['bnf_name'],largest_growth['script_growth'],largest_growth['2016_total']))
last_50_result =list(zip(largest_shrinkage['bnf_name'],largest_shrinkage['script_growth'],largest_shrinkage['2016_total']))
final_result = first_50_result+last_50_result

In [51]:
# script_growth = [("Butec_Transdermal Patch 5mcg/hr", 3.4677419355, 62.0)] * 100
script_growth = final_result

In [52]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [64]:
scripts_q6_df = scripts.copy()
scripts_q6_df.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [65]:
# p = 1/(chem['CHEM SUB'].nunique())
p = 1/(scripts_q6_df['bnf_code'].nunique())
rates = (scripts_q6_df.groupby('bnf_code')['bnf_code'].count())/scripts_q6_df['practice'].count()
rates

bnf_code
0101010C0    0.000035
0101010F0    0.000002
0101010G0    0.000385
0101010I0    0.000035
0101010J0    0.000042
               ...   
239634596    0.000089
239641097    0.000100
239648096    0.000024
239656096    0.000051
239659096    0.000014
Name: bnf_code, Length: 1975, dtype: float64

In [66]:
scripts_q6_df = pd.merge(scripts_q6_df,rates,left_on='bnf_code',right_index=True)
scripts_q6_df.head()

,bnf_code,practice,bnf_code_x,bnf_name,items,nic,act_cost,quantity,bnf_code_y
0,0106020C0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0.001064
93,0106020C0,N81013,0106020C0,Bisacodyl_Tab E/C 5mg,19,28.10,28.14,860,0.001064
1579,0106020C0,N81029,0106020C0,Bisacodyl_Tab E/C 5mg,50,67.73,67.51,2074,0.001064
1580,0106020C0,N81029,0106020C0,Bisacodyl_Suppos 10mg,1,1.77,1.75,6,0.001064
1581,0106020C0,N81029,0106020C0,Dulcolax_Tab 5mg,1,3.78,3.61,56,0.001064


In [27]:
rare_codes =0

In [67]:
scripts_q6_df['rare'] = scripts_q6_df['bnf_code_y']<(0.1*p)

In [68]:
scripts_q6_df.rename(columns={'bnf_code_y':'rates'},inplace=True)

In [69]:
scripts_q6_df[scripts_q6_df['rare']].shape

(11570, 10)

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [70]:
rare_costs = scripts_q6_df[scripts_q6_df.rare == True].groupby(['practice','rare'])['act_cost'].sum()
rare_total_cost = sum(rare_costs)
total_act_cost = scripts_q6_df['act_cost'].sum()
overall_rare_cost_prop = rare_total_cost/total_act_cost
overall_rare_cost_prop

0.015962901360950382

In [39]:
# def find_cost(groupby_object):
#     return groupby_object.groupby('rare')['act_cost'].sum()
# rare_cost_per_practice= scripts_q6_df.groupby('practice').apply(find_cost)
# rare_cost_per_practice

In [71]:
act_cost_per_practice =scripts_q6_df.groupby('practice')['act_cost'].sum()
rare_cost_per_practice = scripts_q6_df[scripts_q6_df.rare == True].groupby('practice')['act_cost'].sum()
costs_df = pd.merge(act_cost_per_practice,rare_cost_per_practice,left_index=True,right_index=True,how='left')
costs_df ['rare_cost_prop'] = costs_df['act_cost_y']/costs_df['act_cost_x']
costs_df['rare_cost_prop'].fillna(0,inplace=True)

In [72]:
rare_cost_prop = costs_df['rare_cost_prop']

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [73]:
relative_rare_cost_prop = rare_cost_prop-overall_rare_cost_prop

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [74]:
standard_errors = relative_rare_cost_prop.std()
standard_errors

0.06050888706745139

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [76]:
rare_scores = relative_rare_cost_prop/standard_errors
rare_scores

practice
A81005   -0.065216
A81007   -0.125308
A81011   -0.179263
A81012   -0.036615
A81017   -0.142190
            ...   
Y05570   -0.263811
Y05583   -0.263811
Y05597   -0.263811
Y05660   -0.258473
Y05670   -0.263811
Name: rare_cost_prop, Length: 856, dtype: float64

In [97]:
practice_names_q6 = practices.groupby('code')['name'].min()
practice_names_q6

code
A81001                       THE DENSHAM SURGERY
A81002                QUEENS PARK MEDICAL CENTRE
A81003                 VICTORIA MEDICAL PRACTICE
A81004                   BLUEBELL MEDICAL CENTRE
A81005                        SPRINGWOOD SURGERY
                           ...                  
Y05758                       VC SKELMERSDALE UCC
Y05762                    WEST MERTON ACCESS HUB
Y05763             ELY LOCAL URGENT CARE SERVICE
Y05764                       MINOR INJURIES UNIT
Y05771    URGENT CARE CENTRE NORTHWICK PARK HOSP
Name: name, Length: 10843, dtype: object

In [102]:
# We can finally put all the data together
final_data_df = pd.DataFrame({'name':practice_names_q6,'z-scores':rare_scores})
final_data_df.reset_index(level=0,inplace=True)
final_data_df.sort_values(by=['z-scores'],inplace=True,ascending=False)
final_data_df=final_data_df.head(100)
final_data_df.shape

(100, 3)

In [103]:
rare_scripts = list(zip(final_data_df['index'],final_data_df['name'],final_data_df['z-scores']))
rare_scripts
# rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

[('Y03472', 'CONSULTANT DIABETES TEAM', 16.262687124655073),
 ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 15.128648195416869),
 ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.542139356104621),
 ('Y03484', 'DMC COMMUNITY DERMATOLOGY CLINIC', 7.287222200297823),
 ('Y04424', 'DMC HEALTHCARE', 6.838614181432868),
 ('Y00631', 'BINGLEY DERMATOLOGY CLINIC', 5.755997549837739),
 ('A89038', 'BARMSTON MEDICAL CENTRE', 5.664940368539937),
 ('Y01696', 'BASSETLAW HOSPICE OF THE GOOD SHEPHERD', 4.29068517147492),
 ('Y03699', 'OLDHAM DERMATOLOGY SERVICE', 4.103743999157243),
 ('Y02045', 'VERNOVA HEALTHCARE CIC', 3.1807154704143983),
 ('Y02823', 'DMC VICARAGE LANE', 3.054121649498702),
 ('Y00997', 'COMMUNITY DERMATOLOGY SERVICE', 3.0181283595352766),
 ('Y05019', 'OLDHAM TOTAL SKIN SERVICE', 2.7247515958989283),
 ('Y01003', 'BURY OOH', 2.350436347139857),
 ('Y05431', 'NOTTS APPLIANCE MANAGEMENT SERVICE', 2.037158543233387),
 ('Y03557', 'IPU 10-17 RECOVERY COVENTRY', 2.0304495001187792),
 ('Y05419',

In [104]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*